In [1]:
import pandas as pd
import numpy as np

In [3]:
dataset=pd.read_csv("Placement.csv")
dataset

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0
211,212,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0
212,213,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0
213,214,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0


In [5]:
def qualQuan(dataset):
    qual=[]
    quan=[]
    for cn in dataset.columns:
        if(dataset[cn].dtype == 'O'):
            qual.append(cn)
        else:
            quan.append(cn)
    return qual, quan

In [7]:
qual,quan=qualQuan(dataset)
print(qual)
print(quan)

['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation', 'status']
['sl_no', 'ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p', 'salary']


In [9]:
def univeriate(dataset,quan):
    descriptive=pd.DataFrame(index=["Mean","Median","Mode",
                                "Q1:25%","Q2:50%","Q3:75%","Q4:100%",
                                "IQR","1.5Rule","Lesser Outlier","Greater Outlier","Min","Max"],columns=quan)
    for cn in quan:
        descriptive.loc["Mean",[cn]]=dataset[cn].mean()
        descriptive.loc["Median",[cn]]=dataset[cn].median()
        descriptive.loc["Mode",[cn]]=dataset[cn].mode()[0]
        descriptive.loc["Q1:25%",[cn]]=dataset.describe().loc["25%",[cn]]
        descriptive.loc["Q2:50%",[cn]]=dataset.describe().loc["50%",[cn]]
        descriptive.loc["Q3:75%",[cn]]=dataset.describe().loc["75%",[cn]]
        descriptive.loc["Q4:100%",[cn]]=dataset.describe().loc["max",[cn]]
        descriptive.loc["IQR",[cn]]=descriptive.loc["Q3:75%",[cn]]-descriptive.loc["Q1:25%",[cn]]
        descriptive.loc["1.5Rule",[cn]]=1.5*descriptive.loc["IQR",[cn]]
        descriptive.loc["Lesser Outlier",[cn]]=descriptive.loc["Q1:25%",[cn]]-1.5*descriptive.loc["IQR",[cn]]
        descriptive.loc["Greater Outlier",[cn]]=descriptive.loc["Q3:75%",[cn]]+1.5*descriptive.loc["IQR",[cn]]
        descriptive.loc["Min",[cn]]=dataset.describe().loc["min",[cn]]
        descriptive.loc["Max",[cn]]=dataset.describe().loc["max",[cn]]
    
    return descriptive

In [11]:
descriptive = univeriate(dataset,quan)
descriptive

,sl_no,ssc_p,hsc_p,degree_p,etest_p,mba_p,salary
Mean,108.0,67.303395,66.333163,66.370186,72.100558,62.278186,288655.405405
Median,108.0,67.0,65.0,66.0,71.0,62.0,265000.0
Mode,1,62.0,63.0,65.0,60.0,56.7,300000.0
Q1:25%,54.5,60.6,60.9,61.0,60.0,57.945,240000.0
Q2:50%,108.0,67.0,65.0,66.0,71.0,62.0,265000.0
Q3:75%,161.5,75.7,73.0,72.0,83.5,66.255,300000.0
Q4:100%,215.0,89.4,97.7,91.0,98.0,77.89,940000.0
IQR,107.0,15.1,12.1,11.0,23.5,8.31,60000.0
1.5Rule,160.5,22.65,18.15,16.5,35.25,12.465,90000.0
Lesser Outlier,-106.0,37.95,42.75,44.5,24.75,45.48,150000.0


In [13]:
def findoutlier(descriptive):
    lesser=[]
    greater=[]
    for cn in quan:
        if(descriptive[cn]["Min"] < descriptive[cn]["Lesser Outlier"]):
            lesser.append(cn)
        if(descriptive[cn]["Max"]> descriptive[cn]["Greater Outlier"]):
            greater.append(cn)
    return lesser, greater

In [16]:
lesser, greater = findoutlier(descriptive)
print(lesser)
print(greater)

['hsc_p']
['hsc_p', 'degree_p', 'salary']


In [20]:
def replceoutlier(dataset):
    descriptive = univeriate(dataset,quan)
    lesser, greater = findoutlier(descriptive)
    for cn in lesser:
        lo=pd.to_numeric(descriptive.loc["Lesser Outlier",[cn]])
        lo=float(lo)
        dataset.loc[dataset[cn]<=lo,cn]= lo
    for cn in greater:
        go=pd.to_numeric(descriptive.loc["Greater Outlier",[cn]])
        go=float(go)
        dataset.loc[dataset[cn]>=go,cn]= go
    return dataset


In [22]:
dataset = replceoutlier(dataset)

C:\Users\USER\AppData\Local\Temp\ipykernel_17504\1292125919.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lo=float(lo)
C:\Users\USER\AppData\Local\Temp\ipykernel_17504\1292125919.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  go=float(go)
C:\Users\USER\AppData\Local\Temp\ipykernel_17504\1292125919.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  go=float(go)
C:\Users\USER\AppData\Local\Temp\ipykernel_17504\1292125919.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  go=float(go)


In [24]:
descriptive = univeriate(dataset,quan)
descriptive

,sl_no,ssc_p,hsc_p,degree_p,etest_p,mba_p,salary
Mean,108.0,67.303395,66.334744,66.358558,72.100558,62.278186,277648.648649
Median,108.0,67.0,65.0,66.0,71.0,62.0,265000.0
Mode,1,62.0,63.0,65.0,60.0,56.7,300000.0
Q1:25%,54.5,60.6,60.9,61.0,60.0,57.945,240000.0
Q2:50%,108.0,67.0,65.0,66.0,71.0,62.0,265000.0
Q3:75%,161.5,75.7,73.0,72.0,83.5,66.255,300000.0
Q4:100%,215.0,89.4,91.15,88.5,98.0,77.89,390000.0
IQR,107.0,15.1,12.1,11.0,23.5,8.31,60000.0
1.5Rule,160.5,22.65,18.15,16.5,35.25,12.465,90000.0
Lesser Outlier,-106.0,37.95,42.75,44.5,24.75,45.48,150000.0


In [26]:
lesser, greater = findoutlier(descriptive)
print(lesser)
print(greater)

[]
[]


In [28]:
dataset['ssc_p']

0      67.00
1      79.33
2      65.00
3      56.00
4      85.80
       ...  
210    80.60
211    58.00
212    67.00
213    74.00
214    62.00
Name: ssc_p, Length: 215, dtype: float64

In [30]:
dataset['ssc_p'].value_counts()

ssc_p
62.00    11
63.00    10
67.00     9
52.00     9
73.00     9
         ..
69.70     1
80.92     1
83.00     1
86.50     1
80.60     1
Name: count, Length: 103, dtype: int64

In [32]:
dataset['ssc_p'].value_counts().index

Index([ 62.0,  63.0,  67.0,  52.0,  73.0,  74.0,  65.0,  69.0,  77.0,  58.0,
       ...
        81.7,  64.6,  52.6, 84.86,  75.0,  69.7, 80.92,  83.0,  86.5,  80.6],
      dtype='float64', name='ssc_p', length=103)

In [34]:
dataset['ssc_p'].value_counts().values

array([11, 10,  9,  9,  9,  7,  7,  7,  6,  6,  4,  4,  4,  4,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  2,  2,  2,  2,  2,  2,  2,  2,  2,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1], dtype=int64)

In [36]:
dataset['ssc_p'].value_counts().sort_values

<bound method Series.sort_values of ssc_p
62.00    11
63.00    10
67.00     9
52.00     9
73.00     9
         ..
69.70     1
80.92     1
83.00     1
86.50     1
80.60     1
Name: count, Length: 103, dtype: int64>

In [38]:
dataset['ssc_p'].value_counts()

ssc_p
62.00    11
63.00    10
67.00     9
52.00     9
73.00     9
         ..
69.70     1
80.92     1
83.00     1
86.50     1
80.60     1
Name: count, Length: 103, dtype: int64

In [40]:
freq_table= pd.DataFrame(columns=['unquevalue','frequency','relative_frequency','cusum'])
freq_table

,unquevalue,frequency,relative_frequency,cusum


In [42]:
freq_table['unquevalue']=dataset["ssc_p"].value_counts().index

In [44]:
freq_table['unquevalue']=dataset["ssc_p"].value_counts().index
freq_table['frequency']=dataset["ssc_p"].value_counts().values
freq_table

,unquevalue,frequency,relative_frequency,cusum
0,62.00,11,NaN,NaN
1,63.00,10,NaN,NaN
2,67.00,9,NaN,NaN
3,52.00,9,NaN,NaN
4,73.00,9,NaN,NaN
...,...,...,...,...
98,69.70,1,NaN,NaN
99,80.92,1,NaN,NaN
100,83.00,1,NaN,NaN
101,86.50,1,NaN,NaN


In [46]:
freq_table.info

<bound method DataFrame.info of      unquevalue  frequency relative_frequency cusum
0         62.00         11                NaN   NaN
1         63.00         10                NaN   NaN
2         67.00          9                NaN   NaN
3         52.00          9                NaN   NaN
4         73.00          9                NaN   NaN
..          ...        ...                ...   ...
98        69.70          1                NaN   NaN
99        80.92          1                NaN   NaN
100       83.00          1                NaN   NaN
101       86.50          1                NaN   NaN
102       80.60          1                NaN   NaN

[103 rows x 4 columns]>

In [48]:
freq_table.shape[0]

103

In [50]:
len(freq_table)

103

In [52]:
freq_table['unquevalue']=dataset["ssc_p"].value_counts().index
freq_table['frequency']=dataset["ssc_p"].value_counts().values
freq_table['relative_frequency']=freq_table["frequency"]/(len(freq_table))
freq_table['cusum']=freq_table["relative_frequency"].cumsum()
freq_table

,unquevalue,frequency,relative_frequency,cusum
0,62.00,11,0.106796,0.106796
1,63.00,10,0.097087,0.203883
2,67.00,9,0.087379,0.291262
3,52.00,9,0.087379,0.378641
4,73.00,9,0.087379,0.466019
...,...,...,...,...
98,69.70,1,0.009709,2.048544
99,80.92,1,0.009709,2.058252
100,83.00,1,0.009709,2.067961
101,86.50,1,0.009709,2.077670


In [54]:
# Create function of freq_table
def frequency_table(dataset,cn):
    freq_table= pd.DataFrame(columns=['unquevalue','frequency','relative_frequency','cusum'])
    freq_table['unquevalue']=dataset[cn].value_counts().index
    freq_table['frequency']=dataset[cn].value_counts().values
    freq_table['relative_frequency']=freq_table["frequency"]/(len(freq_table))
    freq_table['cusum']=freq_table["relative_frequency"].cumsum()
    return freq_table

In [56]:
frequency_table(dataset,'ssc_p')

,unquevalue,frequency,relative_frequency,cusum
0,62.00,11,0.106796,0.106796
1,63.00,10,0.097087,0.203883
2,67.00,9,0.087379,0.291262
3,52.00,9,0.087379,0.378641
4,73.00,9,0.087379,0.466019
...,...,...,...,...
98,69.70,1,0.009709,2.048544
99,80.92,1,0.009709,2.058252
100,83.00,1,0.009709,2.067961
101,86.50,1,0.009709,2.077670
